In [1]:
import smbus
import time

class mpu6050:
    GRAVITIY_MS2 = 9.80665
    address = None
    bus = None

    ACCEL_SCALE_MODIFIER_2G = 16384.0
    ACCEL_SCALE_MODIFIER_4G = 8192.0
    ACCEL_SCALE_MODIFIER_8G = 4096.0
    ACCEL_SCALE_MODIFIER_16G = 2048.0

    GYRO_SCALE_MODIFIER_250DEG = 131.0
    GYRO_SCALE_MODIFIER_500DEG = 65.5
    GYRO_SCALE_MODIFIER_1000DEG = 32.8
    GYRO_SCALE_MODIFIER_2000DEG = 16.4

    ACCEL_RANGE_2G = 0x00
    ACCEL_RANGE_4G = 0x08
    ACCEL_RANGE_8G = 0x10
    ACCEL_RANGE_16G = 0x18

    GYRO_RANGE_250DEG = 0x00
    GYRO_RANGE_500DEG = 0x08
    GYRO_RANGE_1000DEG = 0x10
    GYRO_RANGE_2000DEG = 0x18

    FILTER_BW_256=0x00
    FILTER_BW_188=0x01
    FILTER_BW_98=0x02
    FILTER_BW_42=0x03
    FILTER_BW_20=0x04
    FILTER_BW_10=0x05
    FILTER_BW_5=0x06

    PWR_MGMT_1 = 0x6B
    PWR_MGMT_2 = 0x6C

    ACCEL_XOUT0 = 0x3B
    ACCEL_YOUT0 = 0x3D
    ACCEL_ZOUT0 = 0x3F

    TEMP_OUT0 = 0x41

    GYRO_XOUT0 = 0x43
    GYRO_YOUT0 = 0x45
    GYRO_ZOUT0 = 0x47

    ACCEL_CONFIG = 0x1C
    GYRO_CONFIG = 0x1B
    MPU_CONFIG = 0x1A

    def __init__(self, address, bus=1):
        self.address = address
        self.bus = smbus.SMBus(bus)
        self.bus.write_byte_data(self.address, self.PWR_MGMT_1, 0x00)

    def read_i2c_word(self, register):
        high = self.bus.read_byte_data(self.address, register)
        low = self.bus.read_byte_data(self.address, register + 1)
        value = (high << 8) + low
        return value - 65536 if value >= 0x8000 else value

    def get_temp(self):
        raw_temp = self.read_i2c_word(self.TEMP_OUT0)
        return (raw_temp / 340.0) + 36.53

    def read_accel_range(self, raw = False):
        raw_data = self.bus.read_byte_data(self.address, self.ACCEL_CONFIG)
        if raw: return raw_data
        return {self.ACCEL_RANGE_2G: 2, self.ACCEL_RANGE_4G: 4, self.ACCEL_RANGE_8G: 8, self.ACCEL_RANGE_16G: 16}.get(raw_data, -1)

    def get_accel_data(self, g = False):
        x = self.read_i2c_word(self.ACCEL_XOUT0)
        y = self.read_i2c_word(self.ACCEL_YOUT0)
        z = self.read_i2c_word(self.ACCEL_ZOUT0)
        accel_range = self.read_accel_range(True)
        modifier = {
            self.ACCEL_RANGE_2G: self.ACCEL_SCALE_MODIFIER_2G,
            self.ACCEL_RANGE_4G: self.ACCEL_SCALE_MODIFIER_4G,
            self.ACCEL_RANGE_8G: self.ACCEL_SCALE_MODIFIER_8G,
            self.ACCEL_RANGE_16G: self.ACCEL_SCALE_MODIFIER_16G,
        }.get(accel_range, self.ACCEL_SCALE_MODIFIER_2G)
        x, y, z = x/modifier, y/modifier, z/modifier
        if g:
            return {'x': x, 'y': y, 'z': z}
        else:
            return {'x': x*self.GRAVITIY_MS2, 'y': y*self.GRAVITIY_MS2, 'z': z*self.GRAVITIY_MS2}

    def read_gyro_range(self, raw = False):
        raw_data = self.bus.read_byte_data(self.address, self.GYRO_CONFIG)
        if raw: return raw_data
        return {self.GYRO_RANGE_250DEG: 250, self.GYRO_RANGE_500DEG: 500,
                self.GYRO_RANGE_1000DEG: 1000, self.GYRO_RANGE_2000DEG: 2000}.get(raw_data, -1)

    def get_gyro_data(self):
        x = self.read_i2c_word(self.GYRO_XOUT0)
        y = self.read_i2c_word(self.GYRO_YOUT0)
        z = self.read_i2c_word(self.GYRO_ZOUT0)
        gyro_range = self.read_gyro_range(True)
        modifier = {
            self.GYRO_RANGE_250DEG: self.GYRO_SCALE_MODIFIER_250DEG,
            self.GYRO_RANGE_500DEG: self.GYRO_SCALE_MODIFIER_500DEG,
            self.GYRO_RANGE_1000DEG: self.GYRO_SCALE_MODIFIER_1000DEG,
            self.GYRO_RANGE_2000DEG: self.GYRO_SCALE_MODIFIER_2000DEG,
        }.get(gyro_range, self.GYRO_SCALE_MODIFIER_250DEG)
        return {'x': x/modifier, 'y': y/modifier, 'z': z/modifier}

    def get_all_data(self):
        return {
            'temperature': self.get_temp(),
            'accel': self.get_accel_data(),
            'gyro': self.get_gyro_data()
        }


In [2]:
# Tạo file rỗng tên là data_imu.txt
open("data_imu_chinh.txt", "w").close()


In [ ]:
import time
import math
import datetime


sensor = mpu6050(0x68, bus=0)

# Tỉ lệ lọc (giữa gyro và accel)
alpha = 0.9

pitch = 0
roll = 0
dt = 0.05  # Thời gian giữa 2 lần đọc, đơn vị: giây

# Ghi tiêu đề (nếu lần đầu ghi)
with open("data_imu_chinh.txt", "w") as f:
    f.write(f"{'Time':<10}  {'Ax':<10}{'Ay':<10}{'Az':<10}{'Gx':<10}{'Gy':<10}{'Gz':<10}{'Pitch':<10}{'Roll':<10}\n")

while True:
    accel_data = sensor.get_accel_data()
    gyro_data = sensor.get_gyro_data()

    # Lấy dữ liệu
    ax, ay, az = accel_data['x'], accel_data['y'], accel_data['z']
    gx, gy, gz = gyro_data['x'], gyro_data['y'], gyro_data['z']

    # Tính pitch/roll từ gia tốc kế (thô)
    accel_pitch = math.degrees(math.atan2(ay, math.sqrt(ax*ax + az*az)))
    accel_roll = math.degrees(math.atan2(-ax, az))

    # Tích phân gyro để lấy góc quay
    pitch = alpha * (pitch + gy * dt) + (1 - alpha) * accel_pitch
    roll = alpha * (roll + gx * dt) + (1 - alpha) * accel_roll

    # Lấy thời gian hiện tại dưới dạng HH:MM:SS
    current_time = datetime.datetime.now().strftime('%H:%M:%S')

    # Hiển thị lên màn hình
    print(f"[{current_time}]  Ax={ax:.2f}, Ay={ay:.2f}, Az={az:.2f} | Gx={gx:.2f}, Gy={gy:.2f}, Gz={gz:.2f} | Pitch={pitch:.2f}, Roll={roll:.2f}")

    # Ghi dữ liệu
    with open("data_imu.txt", "a") as f:
        f.write(f"{current_time:<10}  {ax:<10.2f}{ay:<10.2f}{az:<10.2f}{gx:<10.2f}{gy:<10.2f}{gz:<10.2f}{pitch:<10.2f}{roll:<10.2f}\n")

    time.sleep(dt)


In [ ]:
with open("data_imu.txt", "r") as f:
    content = f.read()
    print(content)


In [ ]:
with open("data_imu.txt", "w"):
    pass

In [6]:
class Kalman1D:
#     xe chạy nhanh thì tên tinh chỉnh 
#     Q_angle = 0.002      # thấp hơn một chút để gyro mượt
#     Q_bias = 0.003
#     R_measure = 0.1 ~ 0.3   # tăng lên so với lúc đứng yên

    
    def __init__(self, Q_angle=0.5, Q_bias=0.01, R_measure=0.5):    # điều chỉnh giá trị  
        self.Q_angle = Q_angle
        self.Q_bias = Q_bias
        self.R_measure = R_measure

        self.angle = 0.0     # Góc lọc được
        self.bias = 0.0      # Sai số bias của gyro
        self.rate = 0.0

        self.P = [[0, 0], [0, 0]]  # Ma trận hiệp phương sai

    def get_angle(self, new_angle, new_rate, dt):
        # Cập nhật rate từ gyro trừ bias
        self.rate = new_rate - self.bias
        self.angle += dt * self.rate

        # Cập nhật ma trận sai số
        self.P[0][0] += dt * (dt*self.P[1][1] - self.P[1][0] - self.P[0][1] + self.Q_angle)
        self.P[0][1] -= dt * self.P[1][1]
        self.P[1][0] -= dt * self.P[1][1]
        self.P[1][1] += self.Q_bias * dt

        # Tính Kalman Gain
        S = self.P[0][0] + self.R_measure
        K = [self.P[0][0] / S, self.P[1][0] / S]

        # Sai số giữa đo accel và đo tích phân
        y = new_angle - self.angle

        # Cập nhật trạng thái
        self.angle += K[0] * y
        self.bias  += K[1] * y

        # Cập nhật P
        P00_temp = self.P[0][0]
        P01_temp = self.P[0][1]

        self.P[0][0] -= K[0] * P00_temp
        self.P[0][1] -= K[0] * P01_temp
        self.P[1][0] -= K[1] * P00_temp
        self.P[1][1] -= K[1] * P01_temp

        return self.angle


In [13]:
# //giống ở trên

from math import atan2, sqrt, degrees
import time
from datetime import datetime

imu = mpu6050(address=0x68, bus=0)
dt = 1

def get_pitch(ax, ay, az):
    return degrees(atan2(ax, sqrt(ay**2 + az**2)))

def get_roll(ay, az):
    return degrees(atan2(ay, az))

while True:
    data = imu.get_all_data()

    ax = data['accel']['x'] / imu.GRAVITIY_MS2
    ay = data['accel']['y'] / imu.GRAVITIY_MS2
    az = data['accel']['z'] / imu.GRAVITIY_MS2

    gx = data['gyro']['x']
    gy = data['gyro']['y']
    gz = data['gyro']['z']

    pitch = get_pitch(ax, ay, az)
    roll = get_roll(ay, az)

    current_time = datetime.now().strftime("%H:%M:%S")

#     print(" " * 120, end='\r')  # clear dòng cũ nếu quá dài
    print(f"[{current_time}]  Ax={ax:.2f}, Ay={ay:.2f}, Az={az:.2f} | Gx={gx:.2f}, Gy={gy:.2f}, Gz={gz:.2f} | Pitch={pitch:.2f}, Roll={roll:.2f}", end='\r', flush=True)

    time.sleep(dt)

KeyboardInterrupt: 

In [ ]:
# // Trường hợp đứng yên, in ra kết quả 


from math import atan2, sqrt, degrees
import time
from datetime import datetime

imu = mpu6050(address=0x68, bus=0)
dt = 0.1

def get_pitch(ax, ay, az):
    return degrees(atan2(ax, sqrt(ay**2 + az**2)))

def get_roll(ay, az):
    return degrees(atan2(ay, az))

while True:
    data = imu.get_all_data()
    
    # Gia tốc
    ax = data['accel']['x'] / imu.GRAVITIY_MS2
    ay = data['accel']['y'] / imu.GRAVITIY_MS2
    az = data['accel']['z'] / imu.GRAVITIY_MS2

    # Con quay
    gx = data['gyro']['x']
    gy = data['gyro']['y']
    gz = data['gyro']['z']

    # Tính pitch, roll từ gia tốc
    pitch = get_pitch(ax, ay, az)
    roll = get_roll(ay, az)

    # Lấy thời gian hiện tại
    current_time = datetime.now().strftime("%H:%M:%S")

    # In ra kết quả
    print(f"[{current_time}]  Ax={ax:.2f}, Ay={ay:.2f}, Az={az:.2f} | Gx={gx:.2f}, Gy={gy:.2f}, Gz={gz:.2f} | Pitch={pitch:.2f}, Roll={roll:.2f}")

    time.sleep(dt)